In [1]:
import sys
import datetime
import os

In [2]:


# Set up the dictionary structure for possible measurements.
#   C => C/A code measurement  
#   P => P code measurement
#   L => Carrier phase
#   D => Doppler
#   S => Carrier noise density (dB-Hz)
#
#   Numbers indicate the signal band (1 => L1, 2 => L2, 3 => L5)
#   P2 is the P code measurment on the L2 band in compliance with the RINEX
#   2.11 speccification.
''

''

In [3]:
rinFileName = "lama0100.18o"
csvFileName = "lama0100.csv"

#rinFileName = "KLGD2130.08O"
#csvFileName = "KLGD2130.csv"
writeMode = "w"

#csvFields = ["time", "prn", "C1", "P1", "L1", "D1", "S1", "C2", "P2", "L2", 
 # "D2", "S2", "C5", "L5", "D5", "S5"]
rinFields = []

numFields = 0
#print(csvFields)

In [4]:
# numFields - number of columns of data
# Open the rinex file and extract header data
rinFile = open(rinFileName, "r")
csvFile = open(csvFileName, writeMode)
currentLine = rinFile.readline()

# cycle before end of header
while currentLine[60:].rstrip() != "END OF HEADER":
    if currentLine[60:].rstrip() == "# / TYPES OF OBSERV": # Extract header data
        numFields = int(currentLine[1:6])
        
       # print(numFields)
        firstLine='time,'+'N_sat,'
        for j in range(int(numFields/9)):
            #print("j=",j)
            for i in range(9):
                rinFields.append(currentLine[(7+i*6):(12+i*6)].lstrip())
                firstLine=firstLine+currentLine[(7+i*6):(12+i*6)].lstrip()+','
            currentLine = rinFile.readline()
       # print(numFields%9)            
        for i in range(numFields%9):
            rinFields.append(currentLine[(7+i*6):(12+i*6)].lstrip())
            firstLine=firstLine+currentLine[(7+i*6):(12+i*6)].lstrip()+','
            #print(i,currentLine)
    currentLine = rinFile.readline()
print(currentLine)
print(rinFields)
# Удаляем последнюю запятую
firstLine=firstLine[:len(firstLine)-1]+""

csvFile.write(firstLine+"\n")
print(firstLine)
print(len(firstLine),firstLine[len(firstLine)-1])
csvFile.close()

                                                            END OF HEADER

['L1', 'L2', 'P1', 'P2', 'C1']
time,N_sat,L1,L2,P1,P2,C1
(25, '1')


In [5]:
# СОЗДАЕМ СЛОВАРЬ ИЗ СПИСКА ПОЛЕЙ + ВРЕМЯ И N СПУТНИКА
gpsRecord = dict(
  time = '0',
  prn = '0'
)
# create new dict from rinFields 
d = dict.fromkeys(rinFields, 0)
# renew gpsrecord with new dict d 
gpsRecord.update(d)
print(gpsRecord)
# convert dict to list for writing list to csv file
csvFields=list(gpsRecord)
#csvFields=(gpsRecord.keys())
#for i in 
#    csvFields.append(gpsRecord)
print(csvFields)

{'P2': 0, 'P1': 0, 'L2': 0, 'L1': 0, 'time': '0', 'C1': 0, 'prn': '0'}
['P2', 'P1', 'L2', 'L1', 'time', 'C1', 'prn']


In [6]:
# READING data after HEADER
# Open CSV file and add data
writeMode="a"
csvFile = open(csvFileName, writeMode)
# write first string
# Scroll through looking for valid epochs (time and number of satellites)
currentLine = rinFile.readline()
while currentLine != "":
      
    if (currentLine[27:30]) == " 0 ":
        year = int(currentLine[1:3])
        if year < 80: 
            year = 2000 + year
        else: 
            1900 + year
        month = int(currentLine[4:6])
        day = int(currentLine[7:9])
        hour = int(currentLine[10:12])
        mm = int(currentLine[13:15])
        sec = float(currentLine[16:26])
        # время 
        # gpsRecord["time"] = str(datetime.date(year, month, day).isoweekday() \
        #  % 7 * 24 * 3600 + hour * 3600 + mm * 60 + sec)
        timef=datetime.date(year, month, day).isoweekday() \
          // 7  + hour + mm/60. + sec/3600.
        gpsRecord["time"] = str('{:.3f}'.format(timef))   
        # Parse the PRN values for this epoch
        numSats = int(currentLine[30:32]) # число спутников
        #print('numSats=',numSats)
        epochPRN = []
        for j in range(int(numSats / 12)):
        # работает, если numSats>=12    
            for i in range(12):
                epochPRN.append(currentLine[32+i*3:35+i*3])
                #print(i,currentLine[32+i*3:35+i*3])
        #    читаем следующию строку для новых спутников > 12
             currentLine = rinFile.readline()
        # работает всегда
        for i in range(numSats % 12):

                epochPRN.append(currentLine[32+i*3:35+i*3])
                #print(i,currentLine[32+i*3:35+i*3])
 
        # parsing epoch
        currentLine = rinFile.readline()
        for prn in epochPRN:
            currentLine = rinFile.readline()
            print(prn,currentLine)
            gpsRecord["prn"] = prn
            for j in range(int(numFields / 5)):
                #currentLine = rinFile.readline()
                for i in range(5):
                    record = currentLine[i*16+1:i*16+14].lstrip()
                    if record == "":
                        gpsRecord[rinFields[j*5+i]] = "0"
                    else:
                        gpsRecord[rinFields[j*5+i]] = record
            if numFields > 5:
                currentLine = rinFile.readline()            #print(numFields)
                for i in range(numFields % 5):
                    #currentLine = rinFile.readline()
                    record = currentLine[i*16+1:i*16+14].lstrip()
                    if record == "":
                        pos=int(numFields/5)*5+i
                        print(pos,i,numFields//5)
                        gpsRecord[rinFields[numFields//5*5+i]] = "0"
                    else:
                        gpsRecord[rinFields[numFields//5*5+i]] = record
                        print(record)
            
            csvLine = ""
            for field in csvFields:
                csvLine += gpsRecord[field] + ","
            csvFile.write(csvLine[:-1]+"\r\n")

    currentLine = rinFile.readline()



IndentationError: unindent does not match any outer indentation level (<ipython-input-6-1d993c00a2c4>, line 36)

In [ ]:
# Close files and clean up
rinFile.close()
csvFile.close()

In [ ]:
datetime.date(year, month, day).isoweekday() \
          // 7  + hour + mm/60. + sec/3600.